In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import datasets

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(multi_class="multinomial"),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [3]:

"""
For filtering base predictors by whether or not they rely on heursitics for multiclass extension

natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
"""

'\nFor filtering base predictors by whether or not they rely on heursitics for multiclass extension\n\nnatively_multi_class_predictors = ["XGBClassifier",\n"BernoulliNB",\n"DecisionTreeClassifier",\n"ExtraTreeClassifier",\n"GaussianNB",\n"KNeighborsClassifier",\n"LabelPropagation",\n"LabelSpreading",\n"LinearDiscriminantAnalysis",\n"LinearSVC", #(setting multi_class=”crammer_singer”)\n"LogisticRegression", #(setting multi_class=”multinomial”)\n"LogisticRegressionCV", #(setting multi_class=”multinomial”)\n"MLPClassifier",\n"NearestCentroid",\n"QuadraticDiscriminantAnalysis",\n"RadiusNeighborsClassifier",\n"RandomForestClassifier",\n"RidgeClassifier",\n"RidgeClassifierCV"]\n\nbase_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}\n'

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2


In [5]:
X_0_train, X_0_test, y_train, y_test = train_test_split(modality_0, y, test_size=0.2, random_state=3, stratify=y)
X_1_train, X_1_test, _,_ = train_test_split(modality_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _,_ = train_test_split(modality_2, y, test_size=0.2, random_state=3, stratify=y)

In [6]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [7]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,  #balanced classes
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )



In [8]:
for name, modality in data_train.items():
    EI.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating meta training data: |          |  0%

Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


...

Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [9]:
EI.meta_training_data

[modality       Modality_0                                                \
 base predictor       ADAB       XGB   DT    RF        GB  KNN        LR   
 sample                  0         0    0     0         0    0         0   
 0                0.993325  0.018859  0.0  0.18  0.247269  0.0  0.494442   
 1                0.993325  1.833657  2.0  0.92  1.383278  1.2  0.597189   
 2                1.496350  1.490131  2.0  1.29  1.483213  1.2  1.206718   
 3                1.496350  1.084872  1.0  1.24  1.170453  0.8  1.312846   
 4                0.993325  1.587603  0.0  0.95  0.999460  1.2  1.071679   
 ..                    ...       ...  ...   ...       ...  ...       ...   
 571              0.993198  0.537792  0.0  0.30  1.337322  0.4  0.362275   
 572              0.993198  0.514297  2.0  0.54  0.475958  0.4  0.944770   
 573              1.496485  1.077167  1.0  1.26  1.072658  1.2  1.170034   
 574              0.993198  0.023123  0.0  0.26  0.149911  0.4  0.466367   
 575        

In [10]:
EI.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |          |  0%

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


In [11]:
EI.meta_summary["metrics"]

,ADAB,XGB,DT,RF,GB,KNN,LR,NB,MLP,SVM
precision,0.926064,0.997245,0.995833,0.994473,0.995885,0.987952,0.989153,0.995839,0.989147,0.994496
recall,0.919444,0.997222,0.995833,0.994444,0.995833,0.987500,0.988889,0.995833,0.988889,0.994444
f1,0.919244,0.997222,0.995830,0.994450,0.995833,0.987555,0.988911,0.995833,0.988914,0.994444


In [12]:
y_pred = EI.predict(X_dict=data_test, meta_model_key="XGB")
y_pred = np.round(y_pred)
y_pred

array([2., 1., 2., 0., 0., 2., 1., 2., 0., 2., 1., 1., 2., 1., 0., 1., 1.,
       2., 1., 1., 0., 0., 2., 0., 1., 1., 1., 1., 2., 0., 2., 2., 0., 2.,
       0., 0., 2., 1., 0., 1., 0., 0., 2., 1., 1., 2., 1., 0., 2., 2., 1.,
       1., 0., 0., 1., 1., 1., 1., 2., 2., 1., 0., 0., 2., 0., 2., 0., 1.,
       0., 1., 1., 2., 2., 0., 1., 2., 1., 1., 2., 0., 0., 0., 2., 1., 0.,
       0., 1., 1., 2., 0., 0., 2., 0., 0., 0., 1., 1., 2., 2., 1., 1., 2.,
       1., 1., 2., 2., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       2., 2., 1., 1., 0., 0., 0., 2., 0., 1., 1., 2., 2., 2., 2., 2., 0.,
       2., 0., 1., 1., 2., 0., 1., 0., 2., 2., 2., 2., 1., 0., 0., 2., 1.,
       1., 2., 1., 1., 2., 0., 2., 0., 1., 0., 0., 2., 2., 0., 2., 2., 2.,
       0., 0., 0., 2., 1., 2., 1., 2., 0., 1.])

In [13]:
y_test

array([2, 1, 2, 0, 0, 2, 1, 2, 0, 2, 1, 1, 2, 1, 0, 1, 1, 2, 1, 1, 0, 0,
       2, 0, 1, 1, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0, 2, 1,
       1, 2, 1, 0, 2, 2, 1, 1, 0, 0, 1, 1, 1, 1, 2, 2, 1, 0, 0, 2, 0, 2,
       0, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 1,
       2, 0, 0, 2, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0, 1, 1, 2, 2,
       2, 2, 2, 0, 2, 0, 1, 1, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 1, 1,
       2, 1, 1, 2, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 1, 2,
       1, 2, 0, 1])

In [14]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred))])/len(y_test)
accuracy # =179/180

0.9944444444444445

In [19]:
EI.meta_summary['thresholds']["XGB"]

precision    0.997222
recall       0.997222
f1           0.997222
Name: XGB, dtype: float64

In [15]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [16]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}
model = base_predictors["SVM"]
model.fit(X_train,y_train)
y_pred  = model.predict_proba(X_test)
y_pred[:5],y_pred[:, 1][:5]

#print(y_pred[:5])
#y_pred = [max(range(len(prob)), key=lambda i: prob[i]) for prob in y_pred] #change all probability vectors to predictions for scoring
#print(y_pred[:5])

(array([[0.00981237, 0.0276197 , 0.96256793],
        [0.00948382, 0.98086462, 0.00965155],
        [0.96882308, 0.0189461 , 0.01223082],
        [0.01033915, 0.00398004, 0.98568081],
        [0.96210977, 0.02622078, 0.01166945]]),
 array([0.0276197 , 0.98086462, 0.0189461 , 0.00398004, 0.02622078]))

In [17]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred, average='macro')

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets